In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from scipy import stats

%matplotlib inline
# plt.style.use('ggplot')

In [3]:
def read_data(file_path, column_names):
    data = pd.read_csv(file_path, header = 0, names = column_names)
    return data

def feature_normalize(dataset):
    mu = np.mean(dataset, axis = 0)
    sigma = np.std(dataset, axis = 0)
    return (dataset - mu) / sigma

def convert_timestamp(dataset):
    dataset['Timestamp'] = dataset['Timestamp'].apply(lambda x:
                                                     datetime.strptime(x, '%m/%d/%y %H:%M').timestamp())
    return dataset

def convert_BGA_RFU(dataset, threshold):
    dataset['BGA_Phycocyanin_RFU'] = dataset['BGA_Phycocyanin_RFU'].apply(lambda x: 1 if x >= threshold else 0)
    return dataset

def plot_axis(ax, x, y, title):
    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(false)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(y), max(x)])
    ax.grid(true)
    
def plot_activity(activity, data):
    fig, (ax0, ax1, ax2) = plt.subplots(nrows = 3, figsize = (15, 10), sharex = True)
    plot_axis(ax0, data['timestamp'], data['x-axis'], 'x-axis')
    plot_axis(ax1, data['timestamp'], data['y-axis'], 'y-axis')
    plot_axis(ax2, data['timestamp'], data['z-axis'], 'z-axis')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()
    
# We step 50% down based on window size
def windows(data, size):
    counter_output = 0
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)
        counter_output += 1
        if counter_output % 10 == 0:
            print('Window Segmentation {0:.2f}% done'.format((start / data.count()) * 100))
    
# 32 chosen for 8 hours of 15 minute intervals
# TODO: NEED TO PASS IN COLUMNS AND DETERMINE FROM THAT
def segment_signal(dataset, window_size = 90, columns = 5):
    segments = np.empty((0, window_size, columns))
    labels = np.empty((0))
    count = 0;
    for (start, end) in windows(dataset["Timestamp"], window_size):
        temperature = dataset['Temperature'][start:end]
        conductivity = dataset['Sp_Cond'][start:end]
        turbidity = dataset['Turbidity'][start:end]
        ph = dataset['pH'][start:end]
        odo = dataset['ODO'][start:end]
        if(len(dataset['Timestamp'][start:end]) == window_size):
            segments = np.vstack([segments, np.dstack([temperature, conductivity, turbidity, ph, odo])])
            labels = np.append(labels, stats.mode(dataset['BGA_Phycocyanin_RFU'][start:end])[0][0])
    return segments, labels

In [4]:
cols = ['Timestamp', 'Temperature', 'Sp_Cond', 'pH_mV', 'pH', 'Turbidity', 'Chlorophyll', 'Chlorophyll_RFU',
        'ODOSat', 'ODO', 'BGA_Phycocyanin_RFU']

cols_to_keep = []
cols_to_keep.append('Timestamp')
cols_to_keep.append('Temperature')
cols_to_keep.append('Sp_Cond')
# cols_to_keep.append('pH_mV')
cols_to_keep.append('pH')
cols_to_keep.append('Turbidity')
# cols_to_keep.append('Chlorophyll')
# cols_to_keep.append('Chlorophyll_RFU')
# cols_to_keep.append('ODOSat')
cols_to_keep.append('ODO')
cols_to_keep.append('BGA_Phycocyanin_RFU')

lake_dataset = read_data('./data/cleaned/utah_lake_vineyard.csv', cols) 

for col in cols:
    if col not in cols_to_keep:
        lake_dataset = lake_dataset.drop(col, axis=1)
        
lake_dataset = convert_timestamp(lake_dataset)
lake_dataset = convert_BGA_RFU(lake_dataset, 1.0)
lake_dataset.head()

,Timestamp,Temperature,Sp_Cond,pH,Turbidity,ODO,BGA_Phycocyanin_RFU
0,1.493960e+09,15.02,1848,8.36,16.84,9.04,0
1,1.493961e+09,14.99,1847,8.36,16.76,9.04,0
2,1.493962e+09,14.96,1847,8.36,16.82,9.04,0
3,1.493963e+09,14.95,1848,8.36,17.19,9.03,0
4,1.493964e+09,14.92,1848,8.36,16.85,9.02,0


In [5]:
window_size = 90

In [6]:
segments, labels = segment_signal(lake_dataset)

Window Segmentation 2.38% done
Window Segmentation 4.75% done
Window Segmentation 7.13% done
Window Segmentation 9.50% done
Window Segmentation 11.88% done
Window Segmentation 14.25% done
Window Segmentation 16.63% done
Window Segmentation 19.00% done
Window Segmentation 21.38% done
Window Segmentation 23.75% done
Window Segmentation 26.13% done
Window Segmentation 28.50% done
Window Segmentation 30.88% done
Window Segmentation 33.25% done
Window Segmentation 35.63% done
Window Segmentation 38.00% done
Window Segmentation 40.38% done
Window Segmentation 42.75% done
Window Segmentation 45.13% done
Window Segmentation 47.50% done
Window Segmentation 49.88% done
Window Segmentation 52.25% done
Window Segmentation 54.63% done
Window Segmentation 57.00% done
Window Segmentation 59.38% done
Window Segmentation 61.75% done
Window Segmentation 64.13% done
Window Segmentation 66.50% done
Window Segmentation 68.88% done
Window Segmentation 71.25% done
Window Segmentation 73.63% done
Window Segme

In [7]:
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

In [8]:
labels.shape

(420, 2)

In [9]:
labels

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1,

In [10]:
lake_dataset.shape

(18947, 7)

In [11]:
segments.shape

(420, 90, 5)

In [12]:
# Args: 
reshaped_segments = segments.reshape(len(segments), 1, window_size, 5)

In [13]:
reshaped_segments.shape

(420, 1, 90, 5)

In [15]:
train_test_split = np.random.rand(len(reshaped_segments)) < 0.70
train_x = reshaped_segments[train_test_split]
train_y = labels[train_test_split]
test_x = reshaped_segments[~train_test_split]
test_y = labels[~train_test_split]

In [17]:
x = tf.placeholder(tf.float32,shape=[None,1,window_size,5])
y_true = tf.placeholder(tf.float32,shape=[None,2])

In [18]:
hold_prob = tf.placeholder(tf.float32)

In [40]:
input_height = 1
input_width = window_size
num_labels = 2
num_channels = 5

batch_size = 10
kernel_size = 60
depth = 60
num_hidden = 500

learning_rate = 0.0001
training_epochs = 100

total_batchs = train_x.shape[0] // batch_size

def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 10, 1],
                          strides=[1, 1, 10, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [41]:
convo_1 = convolutional_layer(x,shape=[5,5,5,90])
convo_1_pooling = max_pool_2by2(convo_1)

In [42]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,90,180])
convo_2_pooling = max_pool_2by2(convo_2)

In [43]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,10*10*180])

In [44]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [45]:
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [46]:
y_pred = normal_full_layer(full_one_dropout,2)

In [47]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [48]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [49]:
init = tf.global_variables_initializer()

In [57]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for epoch in range(10):
        cost_history = np.empty(shape=[1],dtype=float)
        for b in range(total_batchs):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run(train,feed_dict={x: batch_x, y_true: batch_y})
            cost_history = np.append(cost_history,c)
        print ("Epoch: ",epoch," Training Loss: ",np.mean(cost_history)," Training Accuracy: ",
              session.run(accuracy, feed_dict={x: train_x, Y: train_y}))
    
    print ("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))
    
        

#     for i in range(5000):  
#         batch = ch.next_batch(50)
#         sess.run(train, feed_dict={x: batch[0], y_true: batch[1], hold_prob: 0.5})
        
#         # PRINT OUT A MESSAGE EVERY 100 STEPS
#         if i%100 == 0:
            
#             print('Currently on step {}'.format(i))
#             print('Accuracy is:')
#             # Test the Train Model
#             matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

#             acc = tf.reduce_mean(tf.cast(matches,tf.float32))

#             print(sess.run(acc,feed_dict={x:ch.test_images,y_true:ch.test_labels,hold_prob:1.0}))
#             print('\n')
            
# with tf.Session() as session:
#     tf.global_variables_initializer().run()
#     for epoch in range(training_epochs):
#         cost_history = np.empty(shape=[1],dtype=float)
#         for b in range(total_batchs):    
#             offset = (b * batch_size) % (train_y.shape[0] - batch_size)
#             batch_x = train_x[offset:(offset + batch_size), :, :, :]
#             batch_y = train_y[offset:(offset + batch_size), :]
#             _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
#             cost_history = np.append(cost_history,c)
#         print "Epoch: ",epoch," Training Loss: ",np.mean(cost_history)," Training Accuracy: ",
#               session.run(accuracy, feed_dict={X: train_x, Y: train_y})
    
#     print "Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_4' with dtype float
	 [[Node: Placeholder_4 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_4', defined at:
  File "C:\Users\MattS\Anaconda3\envs\tfdeeplearning\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\MattS\Anaconda3\envs\tfdeeplearning\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\MattS\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\MattS\Anaconda3\envs\tfdeeplearning\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\MattS\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\MattS\Anaconda3\envs\tf

# PLAYGROUND

In [ ]:
lake_dataset.head()

In [ ]:
lake_dataset.iloc[0]['Timestamp']

In [ ]:
lake_dataset.head()

In [ ]:
print(datetime.strptime('5/5/17 13:15', '%m/%d/%y %H:%M').timestamp())